In [2]:
import json

GROUP_CONFIG = "../config/tja2.json"

with open(GROUP_CONFIG, "r") as f:
    groups = json.load(f)

def extract_target_tags(groups):
    tags = set()

    for group_items in groups.values():
        for item in group_items:
            target = item.get("target", {})

            # target.single
            tags.update(target.get("single", []))

            # target.avg
            for v in target.get("avg", {}).values():
                tags.update(v)

            # target.delta
            delta = target.get("delta", {})
            for side in ["left", "right"]:
                tags.update(delta.get(side, {}).get("tags", []))

    return sorted(tags)

TARGET_TAGS = extract_target_tags(groups)

print(f"[INFO] Total TARGET tags: {len(TARGET_TAGS)}")

# optional: tampilkan list
for t in TARGET_TAGS:
    print(t)

[INFO] Total TARGET tags: 69
OPC1.U2RO.20DAS0A:FT2A010401.PNT
OPC1.U2RO.20DAS0A:FT2A010503.PNT
OPC1.U2RO.20DAS0A:TC2A011401.PNT
OPC1.U2RO.20DAS0A:TC2A011404.PNT
OPC1.U2RO.20DAS0A:TC2A011405.PNT
OPC1.U2RO.20DAS0A:TC2A011406.PNT
OPC1.U2RO.20DAS0A:TC2A011503.PNT
OPC1.U2RO.20DAS0A:TC2A011504.PNT
OPC1.U2RO.20DAS0A:TC2A011505.PNT
OPC1.U2RO.20DAS0A:TC2A011506.PNT
OPC1.U2RO.20DAS0A:TC2A011507.PNT
OPC1.U2RO.20DAS0A:TC2A011601.PNT
OPC1.U2RO.20DAS0A:TC2A011602.PNT
OPC1.U2RO.20DAS0A:TC2A011603.PNT
OPC1.U2RO.20DAS0A:TC2A011604.PNT
OPC1.U2RO.20DAS0A:TC2A011605.PNT
OPC1.U2RO.20DAS0A:TC2A012101.PNT
OPC1.U2RO.20DAS0A:TC2A012102.PNT
OPC1.U2RO.20DAS0A:TC2A012103.PNT
OPC1.U2RO.20DAS0A:TC2A012104.PNT
OPC1.U2RO.20DAS0A:TC2A012105.PNT
OPC1.U2RO.20DAS0A:TC2A012203.PNT
OPC1.U2RO.20DAS0A:TC2A012204.PNT
OPC1.U2RO.20DAS0A:TC2A012205.PNT
OPC1.U2RO.20DAS0A:TC2A012303.PNT
OPC1.U2RO.20DAS0A:TC2A012305.PNT
OPC1.U2RO.20DAS0A:TC2A012306.PNT
OPC1.U2RO.20DAS0B:TC2B023101.PNT
OPC1.U2RO.20DAS0B:TC2B023102.PNT
OPC1.U2RO.20DA

# TJA1

In [3]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CONFIG
PARQUET_DIR = "../data_parquet_per_day/tja1/"
OUTPUT_DIR  = "../plots_flat_only/UNIT1"

START = "2025-12-01"
END   = "2025-12-16"       

RESAMPLE_FREQ = "1T"
FLAT_EPS = 1e-6

os.makedirs(OUTPUT_DIR, exist_ok=True)

# TAG ENGINEER (ARRAY – UNIK)
ENGINEER_TAGS = [
    "EWS102/10DAS0A:FT1A010401.PNT",
    "EWS102/10DAS0A:FT1A010503.PNT",
    "EWS102/10DAS0A:TC1A011401.PNT",
    "EWS102/10DAS0A:TC1A011404.PNT",
    "EWS102/10DAS0A:TC1A011405.PNT",
    "EWS102/10DAS0A:TC1A011406.PNT",
    "EWS102/10DAS0A:TC1A011503.PNT",
    "EWS102/10DAS0A:TC1A011504.PNT",
    "EWS102/10DAS0A:TC1A011505.PNT",
    "EWS102/10DAS0A:TC1A011506.PNT",
    "EWS102/10DAS0A:TC1A011507.PNT",
    "EWS102/10DAS0A:TC1A011601.PNT",
    "EWS102/10DAS0A:TC1A011602.PNT",
    "EWS102/10DAS0A:TC1A011603.PNT",
    "EWS102/10DAS0A:TC1A011604.PNT",
    "EWS102/10DAS0A:TC1A011605.PNT",
    "EWS102/10DAS0A:TC1A012101.PNT",
    "EWS102/10DAS0A:TC1A012102.PNT",
    "EWS102/10DAS0A:TC1A012103.PNT",
    "EWS102/10DAS0A:TC1A012104.PNT",
    "EWS102/10DAS0A:TC1A012105.PNT",
    "EWS102/10DAS0A:TC1A012203.PNT",
    "EWS102/10DAS0A:TC1A012204.PNT",
    "EWS102/10DAS0A:TC1A012205.PNT",
    "EWS102/10DAS0A:TC1A012303.PNT",
    "EWS102/10DAS0A:TC1A012305.PNT",
    "EWS102/10DAS0A:TC1A012306.PNT",
    "EWS102/10DAS0B:TC1B023101.PNT",
    "EWS102/10DAS0B:TC1B023102.PNT",
    "EWS102/10DAS0B:TC1B023201.PNT",
    "EWS102/10DAS0B:TC1B023202.PNT",
    "EWS102/10DAS0D:PT1D012602.PNT",
    "EWS102/10DAS0D:PT1D012603.PNT",
    "EWS102/10DAS0D:PT1D013602.PNT",
    "EWS102/10DAS0D:PT1D013603.PNT",
    "EWS102/10DAS0D:RT1D012307.PNT",
    "EWS102/10DAS0D:RT1D012706.PNT",
    "EWS102/10DAS0D:RT1D026402.PNT",
    "EWS102/10DAS0D:RT1D026404.PNT",
    "EWS102/10DAS0D:RT1D026406.PNT",
    "EWS102/10DAS0D:TC1D027101.PNT",
    "EWS102/10DAS0D:TC1D027102.PNT",
    "EWS102/10DAS0D:TC1D027201.PNT",
    "EWS102/10DAS0D:TC1D027202.PNT",
    "EWS102/10DAS0D:TC1D027203.PNT",
    "EWS102/10DAS0E:PT1E032601.PNT",
    "EWS102/10DAS0E:PT1E032602.PNT",
    "EWS102/10DAS0E:PT1E032603.PNT",
    "EWS102/10DAS0E:PT1E032604.PNT",
    "EWS102/10DAS0E:PT1E033601.PNT",
    "EWS102/10DAS0E:PT1E033602.PNT",
    "EWS102/10DAS0E:PT1E033603.PNT",
    "EWS102/10DAS0E:PT1E033604.PNT",
    "EWS102/10DAS0E:RT1E032307.PNT",
    "EWS102/10DAS0E:RT1E032706.PNT",
    "EWS102/10DAS0E:RT1E046502.PNT",
    "EWS102/10DAS0E:RT1E046504.PNT",
    "EWS102/10DAS0E:RT1E046506.PNT",
    "EWS102/10DAS0E:TC1E047101.PNT",
    "EWS102/10DAS0E:TC1E047102.PNT",
    "EWS102/10DAS0E:TC1E047103.PNT",
    "EWS102/10DAS0E:TC1E047104.PNT",
    "EWS102/10DAS0E:TC1E047105.PNT",
    "EWS102/10DAS0E:TC1E047201.PNT",
    "EWS102/10DAS0E:TC1E047202.PNT",
    "EWS102/10DAS0E:TC1E047203.PNT",
    "EWS102/10DAS0E:TC1E047204.PNT",
    "EWS102/10DAS0M:TC1M053102.PNT",
    "EWS102/10DISP:STEAMFLOWB.PNT",
]


# LOAD DATA PARQUET
dfs = []
for f in os.listdir(PARQUET_DIR):
    if f.endswith(".parquet"):
        dfs.append(pd.read_parquet(os.path.join(PARQUET_DIR, f)))

df = pd.concat(dfs, ignore_index=True)

df = df.rename(columns={
    "f_date_rec": "timestamp",
    "f_address_no": "tag",
    "f_value": "value"
})

df["timestamp"] = pd.to_datetime(df["timestamp"])
df["value"] = pd.to_numeric(df["value"], errors="coerce")

df = df[
    (df["timestamp"] >= START) &
    (df["timestamp"] < END)
]

# PIVOT TIME SERIES
df_pivot = (
    df.pivot_table(
        index="timestamp",
        columns="tag",
        values="value",
        aggfunc="mean"
    )
    .resample(RESAMPLE_FREQ)
    .mean()
)

# FLAT CHECK (ENGINEER TAGS ONLY)
flat_tags = []

print("\n=== CHECKING FLAT TAGS (ENGINEER LIST ONLY) ===")

for tag in ENGINEER_TAGS:

    if tag not in df_pivot.columns:
        print(f"[SKIP] Tag tidak ada di data: {tag}")
        continue

    s = df_pivot[tag].dropna()

    if s.empty:
        continue

    diff = s.max() - s.min()

    if diff >= FLAT_EPS:
        continue

    flat_tags.append(tag)

    # ---------- PLOT ----------
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(s.index, s.values, linewidth=2)

    ax.set_title(
        f"{tag}\nFLAT (Δ = {diff:.6e})",
        fontsize=11
    )
    ax.set_xlabel("Time")
    ax.set_ylabel("Value")
    ax.grid(True, linestyle="--", alpha=0.4)

    fname = re.sub(r'[\\/:*?"<>|]', "_", tag)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"{fname}.jpg"), dpi=150)
    plt.close()

# REPORT
print(f"\nTotal tag FLAT ditemukan : {len(flat_tags)}")

if flat_tags:
    print("Daftar tag FLAT:")
    for t in flat_tags:
        print(f" - {t}")
else:
    print("Tidak ada tag FLAT pada periode ini")

print(f"\n[OK] Grafik FLAT saja tersimpan di: {OUTPUT_DIR}")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_12668\4209812556.py:123: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .resample(RESAMPLE_FREQ)



=== CHECKING FLAT TAGS (ENGINEER LIST ONLY) ===
[SKIP] Tag tidak ada di data: EWS102/10DAS0D:PT1D012602.PNT
[SKIP] Tag tidak ada di data: EWS102/10DAS0D:PT1D013602.PNT

Total tag FLAT ditemukan : 0
Tidak ada tag FLAT pada periode ini

[OK] Grafik FLAT saja tersimpan di: ../plots_flat_only/UNIT1


# TJA2

In [4]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CONFIG
PARQUET_DIR = "../data_parquet_per_day/tja2/"
OUTPUT_DIR  = "../plots_flat_only/UNIT2"

START = "2025-12-01"
END   = "2025-12-16"       

RESAMPLE_FREQ = "1T"
FLAT_EPS = 1e-6

os.makedirs(OUTPUT_DIR, exist_ok=True)

# TAG ENGINEER (ARRAY – UNIK)
ENGINEER_TAGS = [
    "OPC1.U2RO.20DAS0A:FT2A010401.PNT",
    "OPC1.U2RO.20DAS0A:FT2A010503.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011401.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011404.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011405.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011406.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011503.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011504.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011505.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011506.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011507.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011601.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011602.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011603.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011604.PNT",
    "OPC1.U2RO.20DAS0A:TC2A011605.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012101.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012102.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012103.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012104.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012105.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012203.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012204.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012205.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012303.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012305.PNT",
    "OPC1.U2RO.20DAS0A:TC2A012306.PNT",
    "OPC1.U2RO.20DAS0B:TC2B023101.PNT",
    "OPC1.U2RO.20DAS0B:TC2B023102.PNT",
    "OPC1.U2RO.20DAS0B:TC2B023201.PNT",
    "OPC1.U2RO.20DAS0B:TC2B023202.PNT",
    "OPC1.U2RO.20DAS0D:PT2D012602.PNT",
    "OPC1.U2RO.20DAS0D:PT2D012603.PNT",
    "OPC1.U2RO.20DAS0D:PT2D013602.PNT",
    "OPC1.U2RO.20DAS0D:PT2D013603.PNT",
    "OPC1.U2RO.20DAS0D:RT2D012307.PNT",
    "OPC1.U2RO.20DAS0D:RT2D012706.PNT",
    "OPC1.U2RO.20DAS0D:RT2D026402.PNT",
    "OPC1.U2RO.20DAS0D:RT2D026404.PNT",
    "OPC1.U2RO.20DAS0D:RT2D026406.PNT",
    "OPC1.U2RO.20DAS0D:TC2D027101.PNT",
    "OPC1.U2RO.20DAS0D:TC2D027102.PNT",
    "OPC1.U2RO.20DAS0D:TC2D027201.PNT",
    "OPC1.U2RO.20DAS0D:TC2D027202.PNT",
    "OPC1.U2RO.20DAS0D:TC2D027203.PNT",
    "OPC1.U2RO.20DAS0E:PT2E032601.PNT",
    "OPC1.U2RO.20DAS0E:PT2E032602.PNT",
    "OPC1.U2RO.20DAS0E:PT2E032603.PNT",
    "OPC1.U2RO.20DAS0E:PT2E032604.PNT",
    "OPC1.U2RO.20DAS0E:PT2E033601.PNT",
    "OPC1.U2RO.20DAS0E:PT2E033602.PNT",
    "OPC1.U2RO.20DAS0E:PT2E033603.PNT",
    "OPC1.U2RO.20DAS0E:PT2E033604.PNT",
    "OPC1.U2RO.20DAS0E:RT2E032307.PNT",
    "OPC1.U2RO.20DAS0E:RT2E032706.PNT",
    "OPC1.U2RO.20DAS0E:RT2E046502.PNT",
    "OPC1.U2RO.20DAS0E:RT2E046504.PNT",
    "OPC1.U2RO.20DAS0E:RT2E046506.PNT",
    "OPC1.U2RO.20DAS0E:TC2E047101.PNT",
    "OPC1.U2RO.20DAS0E:TC2E047102.PNT",
    "OPC1.U2RO.20DAS0E:TC2E047103.PNT",
    "OPC1.U2RO.20DAS0E:TC2E047104.PNT",
    "OPC1.U2RO.20DAS0E:TC2E047105.PNT",
    "OPC1.U2RO.20DAS0E:TC2E047201.PNT",
    "OPC1.U2RO.20DAS0E:TC2E047202.PNT",
    "OPC1.U2RO.20DAS0E:TC2E047203.PNT",
    "OPC1.U2RO.20DAS0E:TC2E047204.PNT",
    "OPC1.U2RO.20DAS0M:TC2M053102.PNT",
    "OPC1.U2RO.20DISP:STEAMFLOWB.PNT",
]


# LOAD DATA PARQUET
dfs = []
for f in os.listdir(PARQUET_DIR):
    if f.endswith(".parquet"):
        dfs.append(pd.read_parquet(os.path.join(PARQUET_DIR, f)))

df = pd.concat(dfs, ignore_index=True)

df = df.rename(columns={
    "f_date_rec": "timestamp",
    "f_address_no": "tag",
    "f_value": "value"
})

df["timestamp"] = pd.to_datetime(df["timestamp"])
df["value"] = pd.to_numeric(df["value"], errors="coerce")

df = df[
    (df["timestamp"] >= START) &
    (df["timestamp"] < END)
]

# PIVOT TIME SERIES
df_pivot = (
    df.pivot_table(
        index="timestamp",
        columns="tag",
        values="value",
        aggfunc="mean"
    )
    .resample(RESAMPLE_FREQ)
    .mean()
)

# FLAT CHECK (ENGINEER TAGS ONLY)
flat_tags = []

print("\n=== CHECKING FLAT TAGS (ENGINEER LIST ONLY) ===")

for tag in ENGINEER_TAGS:

    if tag not in df_pivot.columns:
        print(f"[SKIP] Tag tidak ada di data: {tag}")
        continue

    s = df_pivot[tag].dropna()

    if s.empty:
        continue

    diff = s.max() - s.min()

    if diff >= FLAT_EPS:
        continue

    flat_tags.append(tag)

    # ---------- PLOT ----------
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(s.index, s.values, linewidth=2)

    ax.set_title(
        f"{tag}\nFLAT (Δ = {diff:.6e})",
        fontsize=11
    )
    ax.set_xlabel("Time")
    ax.set_ylabel("Value")
    ax.grid(True, linestyle="--", alpha=0.4)

    fname = re.sub(r'[\\/:*?"<>|]', "_", tag)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"{fname}.jpg"), dpi=150)
    plt.close()

# REPORT
print(f"\nTotal tag FLAT ditemukan : {len(flat_tags)}")

if flat_tags:
    print("Daftar tag FLAT:")
    for t in flat_tags:
        print(f" - {t}")
else:
    print("Tidak ada tag FLAT pada periode ini")

print(f"\n[OK] Grafik FLAT saja tersimpan di: {OUTPUT_DIR}")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_12668\3998942802.py:123: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .resample(RESAMPLE_FREQ)



=== CHECKING FLAT TAGS (ENGINEER LIST ONLY) ===

Total tag FLAT ditemukan : 6
Daftar tag FLAT:
 - OPC1.U2RO.20DAS0A:TC2A011604.PNT
 - OPC1.U2RO.20DAS0A:TC2A011605.PNT
 - OPC1.U2RO.20DAS0A:TC2A012104.PNT
 - OPC1.U2RO.20DAS0A:TC2A012105.PNT
 - OPC1.U2RO.20DAS0D:PT2D012602.PNT
 - OPC1.U2RO.20DAS0D:PT2D013602.PNT

[OK] Grafik FLAT saja tersimpan di: ../plots_flat_only/UNIT2
